# faiss+langchain+llm的完整智能问答原型

## 准备工作

In [1]:
%%time
%%capture

!pip install langchain
!pip install langchain-openai
!pip install faiss-gpu
!pip install sentence_transformers
!pip install openai
!pip install python-dotenv

CPU times: user 43.6 ms, sys: 24.9 ms, total: 68.5 ms
Wall time: 14.7 s


In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from openai import OpenAI
import pandas as pd
import numpy as np
import os
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

%load_ext dotenv
%dotenv

token = os.environ.get('ONE_API_TOKEN')
base_url=os.environ.get('ONE_API_URL')

system_message=SystemMessage(content="你是小羽，是一个人工智能助手。")

## 功能实现步骤

### 加载嵌入模型

In [2]:
%%time

model_name='/models/bge-m3'
model_kwargs = {"device": "cuda"}
encode_kwargs = {"normalize_embeddings": True}
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

CPU times: user 3.57 s, sys: 1.32 s, total: 4.89 s
Wall time: 4.36 s


### 加载知识库

一个虚构游戏的人物和故事设定。

In [3]:
data = [[1,'《沧海寄》是一个故事游戏，讲述了公元14世纪至22世纪跨度800余年间，全球范围内，发生的各种故事。涌现出众多人物，在历史的大背景下，合情合理而令人印象深刻的呈现他们的故事。', '故事设定'],
[2,'潘菲洛，14世纪人物，生活在威尼斯。曾经是大商人，破产后又遭遇大瘟疫，出海避险，辗转于黎凡特和加尔各答，重新赢得了财富。晚年定居热那亚。', '人物设定'],
[3,'马努，15世纪人物，是摩鹿加群岛哈朗加帝国的军事贵族，曾经多次联合其他伊斯兰力量，击败葡萄牙和西班牙的入侵。晚年遭遇权力斗争中失败，这也波及到他参与组织的海军阶层。马努死后第5年，西班牙人屠灭了哈朗加帝国。', '人物设定'],
[4,'博尔都，14世纪人物，元末明初人。曾官至南昌府知府。因红巾军战乱，颠沛流离，中年在漠北北元做过枢密院的小官。又因蓝玉扫北被俘，多次辗转后，终老于撒马尔罕。', '人物设定'],
[5,'南宫涛，字澜回，14世纪人物，元末明初人。江南私盐商人家族，早年在泉州及东南亚经商。后因家族关系，协助张士诚起兵，在江浙一带活动。兵败后携家族出海避祸于爪哇。', '人物设定'],
[6,'徐光启，字元简，明朝末期著名的政治家、学者和文学家。早年，在广州结识了耶稣会士郭居静，并被卷入澳门郭居静事件。后出任兵部尚书，在辽东平叛。','人物设定'],
[7,'郭居静，字仰凤，原名拉扎罗。天主教耶稣会意大利籍传教士。明朝时来华传教士之一，是利玛窦传教的主要助手。后卷入郭居静事件，事件平息后，赴上海和杭州开教。卒于杭州。','人物设定'],
[8,'帕尼帕特之战，发生在16世纪20年代，蒙古塔塔尔部经开伯尔山口入侵莫卧儿帝国，在帕尼帕特决战。本书中数十位人物涉及这场战争。','故事设定']]
df = pd.DataFrame(data, columns = ['ID','文本', '分类'])

df

,ID,文本,分类
0,1,《沧海寄》是一个故事游戏，讲述了公元14世纪至22世纪跨度800余年间，全球范围内，发生的各...,故事设定
1,2,潘菲洛，14世纪人物，生活在威尼斯。曾经是大商人，破产后又遭遇大瘟疫，出海避险，辗转于黎凡特...,人物设定
2,3,马努，15世纪人物，是摩鹿加群岛哈朗加帝国的军事贵族，曾经多次联合其他伊斯兰力量，击败葡萄牙...,人物设定
3,4,博尔都，14世纪人物，元末明初人。曾官至南昌府知府。因红巾军战乱，颠沛流离，中年在漠北北元做...,人物设定
4,5,南宫涛，字澜回，14世纪人物，元末明初人。江南私盐商人家族，早年在泉州及东南亚经商。后因家族...,人物设定
5,6,徐光启，字元简，明朝末期著名的政治家、学者和文学家。早年，在广州结识了耶稣会士郭居静，并被卷...,人物设定
6,7,郭居静，字仰凤，原名拉扎罗。天主教耶稣会意大利籍传教士。明朝时来华传教士之一，是利玛窦传教的...,人物设定
7,8,帕尼帕特之战，发生在16世纪20年代，蒙古塔塔尔部经开伯尔山口入侵莫卧儿帝国，在帕尼帕特决战...,故事设定


### 加入索引

In [4]:
contents = df['文本']
contents

0    《沧海寄》是一个故事游戏，讲述了公元14世纪至22世纪跨度800余年间，全球范围内，发生的各...
1    潘菲洛，14世纪人物，生活在威尼斯。曾经是大商人，破产后又遭遇大瘟疫，出海避险，辗转于黎凡特...
2    马努，15世纪人物，是摩鹿加群岛哈朗加帝国的军事贵族，曾经多次联合其他伊斯兰力量，击败葡萄牙...
3    博尔都，14世纪人物，元末明初人。曾官至南昌府知府。因红巾军战乱，颠沛流离，中年在漠北北元做...
4    南宫涛，字澜回，14世纪人物，元末明初人。江南私盐商人家族，早年在泉州及东南亚经商。后因家族...
5    徐光启，字元简，明朝末期著名的政治家、学者和文学家。早年，在广州结识了耶稣会士郭居静，并被卷...
6    郭居静，字仰凤，原名拉扎罗。天主教耶稣会意大利籍传教士。明朝时来华传教士之一，是利玛窦传教的...
7    帕尼帕特之战，发生在16世纪20年代，蒙古塔塔尔部经开伯尔山口入侵莫卧儿帝国，在帕尼帕特决战...
Name: 文本, dtype: object

In [5]:
%%time

# vectors = embeddings.encode(text)
db = FAISS.from_texts(contents, embeddings)
db.index.ntotal

CPU times: user 754 ms, sys: 30.9 ms, total: 785 ms
Wall time: 792 ms


8

### 搜索

In [6]:
%%time

query = "徐光启"
docs = db.similarity_search(query)

docs[0].page_content

CPU times: user 33.1 ms, sys: 3.19 ms, total: 36.3 ms
Wall time: 35.7 ms


'徐光启，字元简，明朝末期著名的政治家、学者和文学家。早年，在广州结识了耶稣会士郭居静，并被卷入澳门郭居静事件。后出任兵部尚书，在辽东平叛。'

In [7]:
%%time

query = "意大利"
docs = db.similarity_search(query,k=3)

for doc in docs:
    display(doc.page_content)

'潘菲洛，14世纪人物，生活在威尼斯。曾经是大商人，破产后又遭遇大瘟疫，出海避险，辗转于黎凡特和加尔各答，重新赢得了财富。晚年定居热那亚。'

'郭居静，字仰凤，原名拉扎罗。天主教耶稣会意大利籍传教士。明朝时来华传教士之一，是利玛窦传教的主要助手。后卷入郭居静事件，事件平息后，赴上海和杭州开教。卒于杭州。'

'《沧海寄》是一个故事游戏，讲述了公元14世纪至22世纪跨度800余年间，全球范围内，发生的各种故事。涌现出众多人物，在历史的大背景下，合情合理而令人印象深刻的呈现他们的故事。'

CPU times: user 30.9 ms, sys: 779 µs, total: 31.7 ms
Wall time: 35.8 ms


### 保存和加载索引

In [8]:
%%time

index_path='faiss-langchain-llm_index'
db.save_local(index_path)

CPU times: user 1.36 ms, sys: 367 µs, total: 1.72 ms
Wall time: 3.71 ms


In [9]:
%%time

db = FAISS.load_local(index_path, 
                 embeddings,
                 allow_dangerous_deserialization=True)
db.index.ntotal

CPU times: user 1.7 ms, sys: 0 ns, total: 1.7 ms
Wall time: 3.03 ms


8

In [10]:
%%time

docs = db.similarity_search('瘟疫')
docs[0].page_content

CPU times: user 26.9 ms, sys: 18.9 ms, total: 45.8 ms
Wall time: 48.2 ms


'潘菲洛，14世纪人物，生活在威尼斯。曾经是大商人，破产后又遭遇大瘟疫，出海避险，辗转于黎凡特和加尔各答，重新赢得了财富。晚年定居热那亚。'

### retriver

In [11]:
%%time

query='摩鹿加'
retriever = db.as_retriever()
docs = retriever.invoke(query)

docs[0].page_content

CPU times: user 34.2 ms, sys: 551 µs, total: 34.8 ms
Wall time: 42 ms


'马努，15世纪人物，是摩鹿加群岛哈朗加帝国的军事贵族，曾经多次联合其他伊斯兰力量，击败葡萄牙和西班牙的入侵。晚年遭遇权力斗争中失败，这也波及到他参与组织的海军阶层。马努死后第5年，西班牙人屠灭了哈朗加帝国。'

### llm对话

In [13]:
%%time

chat = ChatOpenAI(api_key=token, 
                  base_url=base_url,
                  model='xiaoyu', 
                  temperature=0,
                  streaming=True)

messages = [
    system_message,
    HumanMessage(content="徐光启是谁？"),
]

for chunk in chat.stream(messages):
    print(chunk.content, end="", flush=True)

徐光启（1562年11月18日-1633年9月11日），字子先，号玄扈，南直隶松江府上海县人。他是明代著名的科学家、政治家和军事家，尤其在数学、天文、历法、农学等领域有重要贡献。徐光启是明末清初时期科学革命的重要人物，他的《农政全书》是中国古代农学的集大成之作，对后世影响深远。同时，他还参与了《崇祯历书》的编纂工作，试图改革中国的历法体系，以适应日益发展的科技需求。CPU times: user 369 ms, sys: 37.9 ms, total: 407 ms
Wall time: 5.07 s


### 嵌入+llm对话

In [92]:
%%time

from langchain_core.vectorstores import VectorStoreRetriever
from langchain.chains import RetrievalQA

from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

retriever = VectorStoreRetriever(vectorstore=db)

qa_chain = RetrievalQA.from_chain_type(
    llm=chat, retriever=retriever
)

question='简要介绍下徐光启'
qa_chain.invoke({"query": question})

CPU times: user 428 ms, sys: 4.9 ms, total: 433 ms
Wall time: 2.83 s


{'query': '简要介绍下徐光启',
 'result': '徐光启，字元简，明朝末期著名的人物，他是一位政治家、学者和文学家。在年轻时，他与耶稣会士郭居静有交往，并因广州事件而被卷入澳门的郭居静事件。后来，徐光启担任了兵部尚书，在辽东参与平叛活动。'}

In [14]:
%%time

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(chat, prompt)

retriever = db.as_retriever(search_kwargs={"k": 1,"score_threshold": 0.4},search_type="similarity_score_threshold")
retrieval_chain = create_retrieval_chain(retriever, document_chain)

context_docs=None

print('回答: \n')

for chunk in retrieval_chain.stream({"input": "南宫涛是谁？"}):
    if 'answer' in chunk:
        print(chunk['answer'], end="", flush=True)
    elif 'context' in chunk:
        context_docs=chunk['context']
print()

print('\n相似度上下文: \n')
for doc in context_docs:
    print(doc.page_content)
    print()

回答: 

南宫涛是一位生活在14世纪的元末明初人物，他出身于江南的一个私盐商人家族。年轻时，他在泉州和东南亚地区从事商业活动。后来，由于与家族的关系，他参与了张士诚的起义，并在江浙一带活跃。然而，在军事失利后，南宫涛为了躲避战乱，带着家族成员出海逃往爪哇。

相似度上下文: 

南宫涛，字澜回，14世纪人物，元末明初人。江南私盐商人家族，早年在泉州及东南亚经商。后因家族关系，协助张士诚起兵，在江浙一带活动。兵败后携家族出海避祸于爪哇。

CPU times: user 235 ms, sys: 92.2 ms, total: 328 ms
Wall time: 3.5 s


### 删除索引中指定向量记录

### 追加向量记录到索引